### Import Libraries dan Load Data

- Mengimport library yang diperlukan dan memuat dataset train dan test dari file CSV.
- Dataset train berisi 28,000 baris dengan kolom tweet_id, content, dan sentiment.
- Dataset test berisi 12,000 baris dengan kolom tweet_id dan content yang akan diprediksi sentimen-nya. Data ditampilkan untuk memahami struktur awal.

In [50]:
from IPython.display import display
import pandas as pd 

train_df = pd.read_csv("./train.csv")
test_df = pd.read_csv("./test.csv")

print("Data Latihan")
display(train_df.head())

print("Data Uji")
display(test_df.head())

Data Latihan


tweet_id                                            content   sentiment
0  1956967341  @tiffanylue i know  i was listenin to bad habi...       empty
1  1956967666  Layin n bed with a headache  ughhhh...waitin o...     sadness
2  1956967696                Funeral ceremony...gloomy friday...     sadness
3  1956967789               wants to hang out with friends SOON!  enthusiasm
4  1956968416  @dannycastillo We want to trade with someone w...     neutral

Data Uji


tweet_id                                            content
0  1696004910  @AnnebrittB slaying dragons &amp; rescue innoc...
1  1696005032  @reaching4amomnt it'll be close, but hopefully...
2  1696005039  It's teacher appreciation week - been out maki...
3  1696005052  @criminalminds  out of commish for a few days ...
4  1696005125  @PerezHilton hey perez! good luck! well, i'm s...

### Download NLTK Resources

Download resource NLTK yang diperlukan untuk text preprocessing seperti stopwords (kata-kata umum yang tidak bermakna) dan PorterStemmer untuk mengubah kata ke bentuk dasarnya. Resource ini penting untuk membersihkan dan memproses teks sebelum digunakan dalam model.

In [51]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import nltk

nltk.download("stopwords")
nltk.download("wordnet")

stopwords_eng = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

print("NLTK resources berhasil didownload!")

NLTK resources berhasil didownload!


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dvnnfrr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/dvnnfrr/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Text Preprocessing - Fungsi Pembersihan Teks
Membuat fungsi untuk membersihkan dan memproses teks tweet dengan teknik advanced. Fungsi ini melakukan serangkaian langkah preprocessing sebagai berikut:

1. Lowercase Conversion: Mengubah seluruh teks menjadi huruf kecil untuk menghilangkan perbedaan case (misalnya 'Happy' dan 'happy' dianggap sama).
2. URL Removal: Menghapus semua URL yang dimulai dengan 'http' atau 'www' karena tidak mengandung informasi sentimen yang relevan.
3. Mention Removal: Menghapus semua mention pengguna Twitter (format @username) karena tidak berkontribusi pada analisis sentimen.
4. Hashtag Processing: Menghapus simbol '#' namun tetap mempertahankan kata setelahnya, karena kata dalam hashtag sering mengandung informasi sentimen penting.
5. Character Normalization: Menormalisasi karakter berulang (misalnya 'loooove' menjadi 'loove', 'hahahaha' menjadi 'haha') untuk mengurangi variasi kata yang tidak perlu sambil tetap mempertahankan penekanan emosi.
6. Punctuation Preservation: Mempertahankan tanda seru (!) dan tanda tanya (?) karena keduanya merupakan indikator sentimen yang kuat, kemudian menghapus semua karakter selain huruf, spasi, dan kedua tanda baca tersebut.
7. Tokenization: Memecah teks menjadi token-token individual (kata-kata).
8. Stopwords Removal: Menghapus kata-kata umum yang tidak bermakna (seperti 'the', 'is', 'at') menggunakan daftar stopwords bahasa Inggris dari NLTK untuk mengurangi noise dalam data.
9. Stemming: Mengubah kata ke bentuk dasarnya menggunakan Porter Stemmer (misalnya 'running', 'runs', 'ran' menjadi 'run') untuk mengurangi dimensi fitur dan mengelompokkan kata-kata dengan akar yang sama.

Preprocessing ini sangat penting agar model fokus pada kata-kata yang benar-benar bermakna dan mengurangi noise dalam data, sehingga meningkatkan performa klasifikasi sentimen.

In [52]:
def clean_text(text):
    text = text.lower()

    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#", "", text)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    text = re.sub(r"[^a-z!?\s]", " ", text)

    tokens = text.split()
    
    tokens = [t for t in tokens if t not in stopwords_eng]
    tokens = [stemmer.stem(t) for t in tokens]
    
    return " ".join(tokens)

print("Fungsi pembersihan teks berhasil dibuat!")

Fungsi pembersihan teks berhasil dibuat!


### Apply Text Cleaning ke Dataset

Menerapkan fungsi pembersihan teks ke seluruh dataset train. Proses ini membersihkan semua tweet dari noise seperti URL, mention, hashtag, dan karakter khusus. Hasil preprocessing disimpan dalam kolom baru 'clean_content' dan kolom 'content' asli dihapus untuk menghemat memori. Teks yang sudah bersih ini akan digunakan untuk ekstraksi fitur dan training model.

In [53]:
train_df["clean_content"] = train_df["content"].astype(str).apply(clean_text)

print("Text cleaning selesai!")
display(train_df.head())

Text cleaning selesai!


tweet_id                                            content   sentiment  \
0  1956967341  @tiffanylue i know  i was listenin to bad habi...       empty   
1  1956967666  Layin n bed with a headache  ughhhh...waitin o...     sadness   
2  1956967696                Funeral ceremony...gloomy friday...     sadness   
3  1956967789               wants to hang out with friends SOON!  enthusiasm   
4  1956968416  @dannycastillo We want to trade with someone w...     neutral   

                                       clean_content  
0  know listenin bad habit earlier start freakin ...  
1               layin n bed headach ughh waitin call  
2                       funer ceremoni gloomi friday  
3                             want hang friend soon!  
4               want trade someon houston ticket one

### Split Data dan Feature Engineering - TF-IDF

Membagi data menjadi train dan validation set dengan proporsi 92:8 (test_size=0.08), lalu melakukan feature engineering menggunakan TF-IDF (Term Frequency-Inverse Document Frequency). Proses ini dilakukan dengan langkah-langkah berikut:

1. **Data Splitting**: Membagi dataset menjadi training set (92%) dan validation set (8%) menggunakan stratified sampling untuk memastikan proporsi setiap kelas sentimen tetap seimbang di kedua set. Parameter random_state=42 digunakan untuk reproducibility.

2. **Word-level TF-IDF (N-gram 1-3)**: Membuat representasi fitur berbasis kata dengan n-gram range (1,3), yang berarti menangkap unigram (kata tunggal), bigram (dua kata berurutan), dan trigram (tiga kata berurutan). Fitur ini menangkap makna semantik dan konteks kata dalam kalimat. Parameter yang digunakan:
   - max_features=50,000: Membatasi jumlah fitur maksimal untuk menghindari overfitting
   - min_df=3: Mengabaikan term yang muncul kurang dari 3 dokumen
   - max_df=0.90: Mengabaikan term yang muncul di lebih dari 90% dokumen (terlalu umum)
   - sublinear_tf=True: Menggunakan scaling logaritmik pada term frequency
   - norm='l2': Normalisasi L2 untuk menormalkan vektor fitur
   - use_idf=True: Menggunakan inverse document frequency weighting

3. **Character-level TF-IDF (N-gram 3-5)**: Membuat representasi fitur berbasis karakter dengan n-gram range (3,5), yang menangkap pola morfologi, typo, dan variasi ejaan kata. Ini sangat berguna untuk menangani kata-kata informal, singkatan, dan kesalahan ketik yang sering muncul di Twitter. Parameter:
   - analyzer="char": Analisis berbasis karakter, bukan kata
   - max_features=30,000: Membatasi jumlah fitur karakter
   - ngram_range=(3,5): Menangkap sequence 3-5 karakter

4. **Unigram-only TF-IDF**: Membuat representasi fitur khusus untuk kata-kata individual yang kuat (single words only). Ini membantu menangkap kata-kata sentimen yang powerful tanpa tergantung konteks. Parameter:
   - ngram_range=(1,1): Hanya kata tunggal
   - max_features=20,000: Membatasi pada 20,000 kata paling penting
   - min_df=2: Term minimal muncul di 2 dokumen

5. **Fit and Transform**: Melakukan fitting (pembelajaran vocabulary) pada training set, kemudian mentransformasi baik training maupun validation set menjadi representasi numerik TF-IDF. Penting untuk hanya fit pada training set untuk menghindari data leakage.

6. **Feature Combination**: Menggabungkan ketiga jenis fitur (word-level, character-level, dan unigram) menggunakan scipy.sparse.hstack untuk menghasilkan representasi fitur yang lebih kaya dan comprehensive. Kombinasi ini memberikan model informasi dari berbagai perspektif: makna semantik, pola morfologi, dan kata-kata kunci individual.

Hasil akhir adalah matriks fitur sparse yang menggabungkan ketiga representasi TF-IDF, memberikan model informasi maksimal untuk klasifikasi sentimen yang lebih akurat. Total fitur yang dihasilkan adalah kombinasi dari ketiga vectorizer, yang ditampilkan dalam output shape.

In [54]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

X = train_df["clean_content"]
y = train_df["sentiment"]   

X_train_text, X_val_text, y_train, y_val = train_test_split(
    X, y, test_size=0.08, random_state=42, stratify=y
)

tfidf_word = TfidfVectorizer(
    ngram_range=(1, 3),
    max_features=50000,     
    min_df=3,              
    max_df=0.90,          
    sublinear_tf=True,
    norm='l2',
    use_idf=True
)

tfidf_char = TfidfVectorizer(
    analyzer="char",
    ngram_range=(3, 5),      
    max_features=30000, 
    min_df=3,                
    max_df=0.95,
    sublinear_tf=True
)

tfidf_unigram = TfidfVectorizer(
    ngram_range=(1, 1),      
    max_features=20000,
    min_df=2,
    max_df=0.95,
    sublinear_tf=True,
    norm='l2'
)

# Fit and transform
X_train_word = tfidf_word.fit_transform(X_train_text)
X_val_word   = tfidf_word.transform(X_val_text)

X_train_char = tfidf_char.fit_transform(X_train_text)
X_val_char   = tfidf_char.transform(X_val_text)

X_train_uni = tfidf_unigram.fit_transform(X_train_text)
X_val_uni   = tfidf_unigram.transform(X_val_text)

# Combine features
X_train_final = hstack([X_train_word, X_train_char, X_train_uni])
X_val_final   = hstack([X_val_word, X_val_char, X_val_uni])

print(f"Feature engineering selesai!")
print(f"Train features shape: {X_train_final.shape}")
print(f"Validation features shape: {X_val_final.shape}")

Feature engineering selesai!
Train features shape: (25760, 49404)
Validation features shape: (2240, 49404)


### Hyperparameter Tuning - Grid Search

Melakukan grid search untuk menemukan kombinasi hyperparameter terbaik pada model Logistic Regression. Proses ini dilakukan dengan langkah-langkah sistematis sebagai berikut:

1. **Import Libraries**: Mengimport library yang diperlukan untuk berbagai algoritma machine learning (Multinomial Naive Bayes, Bernoulli Naive Bayes, SVM, dan Logistic Regression) serta metrik evaluasi F1-Score yang akan digunakan untuk menilai performa model.

2. **Definisi Parameter Grid**: Membuat dictionary parameter grid yang berisi kombinasi hyperparameter yang akan diuji. Dalam kasus ini, fokus pada Logistic Regression dengan:
   - **penalty**: "l2" (L2 regularization/Ridge) untuk mengurangi overfitting dengan cara shrinking weights
   - **solver**: "liblinear" yang cocok untuk dataset kecil-menengah dan mendukung L1/L2 regularization
   - **C**: Parameter regularization strength dengan 13 nilai berbeda [0.01, 0.05, 0.08, 0.085, 0.09, 0.095, 0.1, 0.25, 0.5, 1, 2, 3, 5]. Nilai C yang lebih kecil berarti regularisasi lebih kuat (model lebih simple), nilai C lebih besar berarti regularisasi lebih lemah (model lebih kompleks).

3. **Inisialisasi Storage**: Membuat dictionary kosong `all_results` untuk menyimpan hasil terbaik dari setiap model yang diuji, termasuk F1-score, hyperparameter terbaik, dan model object-nya.

4. **Iterasi Grid Search**: Untuk setiap model dalam parameter grid, dilakukan:
   - **Model Instantiation**: Membuat instance model berdasarkan tipe algoritma (MultinomialNB, BernoulliNB, SVM, atau LogisticRegression) dengan hyperparameter yang sedang diuji
   - **Class Balancing**: Menggunakan `class_weight="balanced"` untuk menangani imbalanced class dengan memberikan weight lebih besar pada kelas minoritas
   - **Training**: Melatih model pada training set (X_train_final, y_train) dengan hyperparameter spesifik
   - **Prediction**: Melakukan prediksi pada validation set (X_val_final)
   - **Evaluation**: Menghitung F1-Score dengan average="macro" yang memberikan bobot sama untuk setiap kelas (tidak terpengaruh imbalance)
   - **Tracking Best Model**: Menyimpan model, score, dan parameter jika menghasilkan F1-Score lebih tinggi dari sebelumnya

5. **Per-Model Results**: Setelah semua kombinasi parameter untuk satu model diuji, menyimpan hasil terbaik (best_score, best_params, best_model) ke dalam dictionary all_results dan menampilkan ringkasan performa model tersebut.

6. **Final Comparison**: Membandingkan semua model yang telah diuji dengan menampilkan F1-Score dan hyperparameter terbaik untuk setiap algoritma dalam format tabel yang mudah dibaca.

7. **Best Model Selection**: Memilih model dengan F1-Score macro tertinggi sebagai model final. Proses ini menggunakan fungsi max() dengan key untuk menemukan model dengan best_score tertinggi, kemudian mengekstrak nama model, score, parameters, dan model object untuk digunakan dalam prediksi test set.

Grid search ini sangat penting untuk menemukan kombinasi hyperparameter optimal yang menghasilkan performa terbaik pada validation set, dengan tujuan akhir adalah model yang dapat generalisasi dengan baik pada data yang belum pernah dilihat (test set). Pendekatan macro F1-Score memastikan bahwa semua kelas sentimen mendapat perhatian yang sama dalam evaluasi, tidak hanya kelas mayoritas.

In [56]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

param_grids = {
    "MultinomialNB": [
        {"alpha": alpha} for alpha in [0.01, 0.05, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
    ],
    "BernoulliNB": [
        {"alpha": alpha} for alpha in [0.01, 0.05, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0]
    ],
    "LogisticRegression": [
        {"penalty": "l2", "solver": "liblinear", "C": c}
        for c in [0.01, 0.05, 0.08, 0.085, 0.09, 0.095, 0.1, 0.25, 0.5, 1, 2, 3, 5]
    ] 
}

all_results = {}

print("=" * 70)
print("HYPERPARAMETER TUNING - COMPARING 3 MODELS")

for model_name, param_grid in param_grids.items():
    print(f"\n{'=' * 70}")
    print(f"Training: {model_name}")
    print(f"{'=' * 70}\n")
    
    best_score = 0
    best_params = None
    best_model = None
    
    for params in param_grid:
        print(f"Testing: {params}", end=" ")
        
        if model_name == "MultinomialNB":
            model = MultinomialNB(**params)
        elif model_name == "BernoulliNB":
            model = BernoulliNB(**params)
        else:  
            model = LogisticRegression(**params, class_weight="balanced", max_iter=8000)
        
        # Train and evaluate
        model.fit(X_train_final, y_train)
        pred = model.predict(X_val_final)
        score = f1_score(y_val, pred, average="macro")
        
        print(f"| F1 = {score:.4f}")
        
        if score > best_score:
            best_score = score
            best_params = params
            best_model = model
    
    all_results[model_name] = {
        "best_score": best_score,
        "best_params": best_params,
        "best_model": best_model
    }
    
    print(f"\n{'─' * 70}")
    print(f"Best {model_name} F1: {best_score:.4f}")
    print(f"Best Params: {best_params}")
    print(f"{'─' * 70}")

print(f"\n{'=' * 70}")
print("FINAL COMPARISON - ALL MODELS")

for model_name, result in all_results.items():
    print(f"{model_name:20s} | F1 = {result['best_score']:.4f} | {result['best_params']}")

best_model_name = max(all_results, key=lambda x: all_results[x]["best_score"])
best_overall_score = all_results[best_model_name]["best_score"]
best_overall_params = all_results[best_model_name]["best_params"]
best_model = all_results[best_model_name]["best_model"]

print(f"\n{'=' * 70}")
print(f"BEST MODEL: {best_model_name}")
print(f"BEST F1 MACRO SCORE: {best_overall_score:.4f}")
print(f"BEST PARAMS: {best_overall_params}")
print(f"{'=' * 70}")

HYPERPARAMETER TUNING - COMPARING 3 MODELS

Training: MultinomialNB

Testing: {'alpha': 0.01} | F1 = 0.1510
Testing: {'alpha': 0.05} | F1 = 0.1682
Testing: {'alpha': 0.1} | F1 = 0.1668
Testing: {'alpha': 0.5} | F1 = 0.1373
Testing: {'alpha': 1.0} | F1 = 0.1200
Testing: {'alpha': 2.0} | F1 = 0.0963
Testing: {'alpha': 5.0} | F1 = 0.0735
Testing: {'alpha': 10.0} | F1 = 0.0627

──────────────────────────────────────────────────────────────────────
Best MultinomialNB F1: 0.1682
Best Params: {'alpha': 0.05}
──────────────────────────────────────────────────────────────────────

Training: BernoulliNB

Testing: {'alpha': 0.01} | F1 = 0.1540
Testing: {'alpha': 0.05} | F1 = 0.1634
Testing: {'alpha': 0.1} | F1 = 0.1662
Testing: {'alpha': 0.5} | F1 = 0.1721
Testing: {'alpha': 1.0} | F1 = 0.1554
Testing: {'alpha': 2.0} | F1 = 0.1391
Testing: {'alpha': 5.0} | F1 = 0.0859
Testing: {'alpha': 10.0} | F1 = 0.0652

──────────────────────────────────────────────────────────────────────
Best BernoulliNB F1

### Prepare Test Set Features - Transformasi Data Test

Mempersiapkan fitur untuk test set menggunakan vectorizer yang sama yang sudah di-fit pada training set. Proses ini dilakukan dengan langkah-langkah berikut:

1. **Text Cleaning pada Test Set**: Menerapkan fungsi `clean_text()` yang sama pada kolom "content" di test_df. Semua tweet dalam test set melalui proses preprocessing identik dengan training set (lowercase, remove URL, mention, hashtag, normalisasi karakter berulang, preserve punctuation, remove stopwords, dan stemming). Konsistensi preprocessing ini sangat krusial untuk memastikan model dapat memproses data test dengan benar.

2. **Word-level TF-IDF Transform**: Mentransformasi teks test yang sudah dibersihkan menggunakan `tfidf_word.transform()` (BUKAN fit_transform). Ini sangat penting karena:
   - Menggunakan vocabulary yang sama dengan training set
   - Tidak membuat vocabulary baru dari test set
   - Mencegah data leakage dan memastikan konsistensi feature space
   - Menghasilkan representasi TF-IDF untuk n-gram (1-3) kata

3. **Character-level TF-IDF Transform**: Mentransformasi teks menggunakan `tfidf_char.transform()` untuk menghasilkan representasi berbasis karakter dengan n-gram (3-5). Sekali lagi menggunakan transform() untuk mempertahankan vocabulary karakter yang sudah dipelajari dari training set, sehingga pola morfologi tetap konsisten.

4. **Unigram TF-IDF Transform**: Mentransformasi teks menggunakan `tfidf_unigram.transform()` untuk mendapatkan representasi kata-kata individual yang kuat. Proses ini juga menggunakan vocabulary unigram yang sudah dipelajari dari training set.

5. **Feature Combination**: Menggabungkan ketiga jenis fitur (word-level, character-level, dan unigram) menggunakan `scipy.sparse.hstack()` untuk membentuk X_test_final. Kombinasi ini harus menghasilkan jumlah kolom (dimensi fitur) yang **identik** dengan X_train_final dan X_val_final, memastikan kompatibilitas dengan model yang sudah dilatih.

6. **Verification**: Menampilkan shape dari X_test_final untuk memverifikasi bahwa jumlah kolom fitur sama dengan training/validation set. Jika dimensi berbeda, model tidak akan bisa melakukan prediksi.

**Catatan Penting**: Seluruh proses ini menggunakan `transform()` bukan `fit_transform()` karena vocabulary, IDF weights, dan feature mapping sudah ditentukan saat training. Menggunakan fit_transform() pada test set akan menyebabkan inconsistency dan hasil prediksi yang tidak valid. Prinsip utamanya adalah: **fit pada training set, transform pada validation dan test set**.

In [57]:
test_df["clean_content"] = test_df["content"].astype(str).apply(clean_text)

# Transform dengan vectorizer yang sudah di-fit
X_test_word = tfidf_word.transform(test_df["clean_content"])
X_test_char = tfidf_char.transform(test_df["clean_content"])
X_test_uni = tfidf_unigram.transform(test_df["clean_content"])

X_test_final = hstack([X_test_word, X_test_char, X_test_uni])

print(f"Test features ready!")
print(f"Test features shape: {X_test_final.shape}")

Test features ready!
Test features shape: (12000, 49404)


### Predict Test Set

Menggunakan model terbaik hasil hyperparameter tuning untuk memprediksi sentimen pada test set. Model akan mengklasifikasikan setiap tweet ke salah satu kategori sentimen berdasarkan pattern yang dipelajari dari training data. Preview prediksi ditampilkan untuk verifikasi sebelum membuat submission file.

In [58]:
test_pred = best_model.predict(X_test_final)

print("Prediksi selesai!")
print("\nPreview prediksi:")
display(pd.DataFrame({
    "tweet_id": test_df["tweet_id"][:10],
    "tweet": test_df["clean_content"][:10],
    "prediction": test_pred[:10]
}))

Prediksi selesai!

Preview prediksi:


tweet_id                                              tweet  prediction
0  1696004910  slay dragon amp rescu innoc could good life mo...  enthusiasm
1  1696005032           close hope figur eat noth cooki week tho     neutral
2  1696005039  teacher appreci week make deliveri instructor ...     neutral
3  1696005052  commish day atl fam saw updat net websit look ...    surprise
4  1696005125  hey perez! good luck! well still plan big pres...        love
5  1696005126  wanna go see edinburgh happen though haha yir ...   happiness
6  1696005128                                   thank that! know   happiness
7  1696005159                 hahahah cours nasti display pictur         fun
8  1696005184                                     good morn love        love
9  1696005222                                  blog entri about?       empty

In [59]:
submission = pd.DataFrame({
    "tweet_id": test_df["tweet_id"],
    "sentiment": test_pred
})

submission.to_csv("./submission.csv", index=False)

print("✓ File submission berhasil dibuat!")
print(f"\nSubmission shape: {submission.shape}")
print("\nSample submission:")
display(submission.head())
print("\nFile disimpan sebagai: ./submission.csv")

✓ File submission berhasil dibuat!

Submission shape: (12000, 2)

Sample submission:


tweet_id   sentiment
0  1696004910  enthusiasm
1  1696005032     neutral
2  1696005039     neutral
3  1696005052    surprise
4  1696005125        love


File disimpan sebagai: ./submission.csv


In [62]:
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime
from wordcloud import WordCloud
from sklearn.metrics import confusion_matrix, classification_report
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet

os.makedirs("./data/plots", exist_ok=True)

# Visualisasi Distribusi Label Sentimen
plt.figure(figsize=(6,4))
train_df["sentiment"].value_counts().sort_index().plot(kind="bar")
plt.title("Distribusi Label Sentimen")
plt.xlabel("Label")
plt.ylabel("Jumlah")
plot_label_dist = "./data/plots/label_distribution.png"
plt.savefig(plot_label_dist, bbox_inches='tight')
plt.close()

# Visualisasi Distribusi Panjang Tweet
train_df["tweet_length"] = train_df["clean_content"].apply(lambda x: len(x.split()))

plt.figure(figsize=(7,4))
plt.hist(train_df["tweet_length"], bins=40)
plt.title("Distribusi Panjang Tweet")
plt.xlabel("Jumlah Kata")
plt.ylabel("Frekuensi")
plot_len_dist = "./data/plots/length_distribution.png"
plt.savefig(plot_len_dist, bbox_inches='tight')
plt.close()

# Confusion Matrix
labels = sorted(train_df["sentiment"].unique())
cm = confusion_matrix(y_val, best_model.predict(X_val_final))

plt.figure(figsize=(6,5))
plt.imshow(cm, cmap="Blues")
plt.title("Confusion Matrix")
plt.colorbar()
plt.xticks(ticks=np.arange(len(labels)), labels=labels, rotation=45)
plt.yticks(ticks=np.arange(len(labels)), labels=labels)

for i in range(len(labels)):
    for j in range(len(labels)):
        plt.text(j, i, cm[i, j], ha="center", va="center")

plot_conf_matrix = "./data/plots/confusion_matrix.png"
plt.tight_layout()
plt.savefig(plot_conf_matrix)
plt.close()

# F1-Score per Kelas
report_dict = classification_report(y_val, best_model.predict(X_val_final), output_dict=True)
f1_scores = [report_dict[str(lbl)]["f1-score"] for lbl in labels]

plt.figure(figsize=(6,4))
plt.bar(labels, f1_scores)
plt.title("F1-Score per Kelas")
plt.ylabel("F1-score")
plt.ylim(0, 1)

plot_f1 = "./data/plots/f1_per_class.png"
plt.tight_layout()
plt.savefig(plot_f1)
plt.close()

# WordCloud per Kelas
wordcloud_paths = {}
for lbl in labels:
    text = " ".join(train_df[train_df["sentiment"] == lbl]["clean_content"])

    wc = WordCloud(
        width=400,
        height=250,
        background_color="white",
        max_words=100
    ).generate(text)

    wc_path = f"./data/plots/wordcloud_{lbl}.png"
    wc.to_file(wc_path)
    wordcloud_paths[lbl] = wc_path

# Top TF-IDF Features per Kelas
feature_names = np.array(
    list(tfidf_word.get_feature_names_out()) +
    list(tfidf_char.get_feature_names_out()) + 
    list(tfidf_unigram.get_feature_names_out())
)

coef = best_model.coef_
top_k = 15
top_features = {}

for idx, lbl in enumerate(labels):
    class_coef = coef[idx]
    pos_idx = np.argsort(class_coef)[-top_k:]
    neg_idx = np.argsort(class_coef)[:top_k]

    top_features[lbl] = {
        "positive": feature_names[pos_idx],
        "negative": feature_names[neg_idx]
    }

pdf_path = "./sentiment_analysis_full_report.pdf"
styles = getSampleStyleSheet()
story = []

story.append(Paragraph("<b>LAPORAN ANALISIS SENTIMEN TWITTER</b>", styles["Title"]))
story.append(Spacer(1, 20))

intro_text = """
Rangkuman lengkap proses analisis sentimen yang meliputi:
- Preprocessing data
- Ekstraksi fitur TF-IDF word-level dan char-level
- Pelatihan model Logistic Regression
- Evaluasi performa model
- Visualisasi dataset dan hasil prediksi
"""
for ln in intro_text.split("\n"):
    story.append(Paragraph(ln, styles["Normal"]))
    story.append(Spacer(1,6))

story.append(Spacer(1,12))

story.append(Paragraph("<b>1. Performa Model</b>", styles["Heading2"]))
story.append(Paragraph(f"F1-Score (Macro): {best_score:.4f}", styles["Normal"]))
story.append(Paragraph(f"Hyperparameter Terbaik: {best_params}", styles["Normal"]))
story.append(Spacer(1, 12))

story.append(Paragraph("<b>2. Ekstraksi Fitur TF-IDF</b>", styles["Heading2"]))
story.append(Paragraph(f"Word-level TF-IDF: {X_train_word.shape[1]} fitur", styles["Normal"]))
story.append(Paragraph(f"Character-level TF-IDF: {X_train_char.shape[1]} fitur", styles["Normal"]))
story.append(Paragraph(f"Total Fitur: {X_train_final.shape[1]} fitur", styles["Normal"]))
story.append(Spacer(1,12))

story.append(Paragraph("<b>3. Informasi Dataset</b>", styles["Heading2"]))
story.append(Paragraph(f"Train samples: {len(X_train_text)}", styles["Normal"]))
story.append(Paragraph(f"Validation samples: {len(X_val_text)}", styles["Normal"]))
story.append(Paragraph(f"Test samples: {len(test_df)}", styles["Normal"]))
story.append(Spacer(1,12))

story.append(Paragraph("<b>4. Visualisasi Dataset</b>", styles["Heading2"]))

story.append(Paragraph("<b>4.1 Distribusi Label Sentimen</b>", styles["Heading3"]))
story.append(Image(plot_label_dist, width=350, height=250))
story.append(Spacer(1,12))

story.append(Paragraph("<b>4.2 Distribusi Panjang Tweet</b>", styles["Heading3"]))
story.append(Image(plot_len_dist, width=350, height=250))
story.append(Spacer(1,12))

story.append(Paragraph("<b>5. Evaluasi Model</b>", styles["Heading2"]))

story.append(Paragraph("<b>5.1 Confusion Matrix</b>", styles["Heading3"]))
story.append(Image(plot_conf_matrix, width=350, height=300))
story.append(Spacer(1,12))

story.append(Paragraph("<b>5.2 F1-Score per Kelas</b>", styles["Heading3"]))
story.append(Image(plot_f1, width=350, height=250))
story.append(Spacer(1,12))

story.append(Paragraph("<b>6. WordCloud Setiap Kelas</b>", styles["Heading2"]))

for lbl in labels:
    story.append(Paragraph(f"<b>Kelas {lbl}</b>", styles["Heading3"]))
    story.append(Image(wordcloud_paths[lbl], width=320, height=200))
    story.append(Spacer(1,10))

story.append(Paragraph("<b>7. Kata Paling Berpengaruh (TF-IDF)</b>", styles["Heading2"]))

for lbl in labels:
    story.append(Paragraph(f"<b>Kelas {lbl}</b>", styles["Heading3"]))
    story.append(Paragraph(
        f"Positive weights: {', '.join(top_features[lbl]['positive'])}",
        styles["Normal"]
    ))
    story.append(Paragraph(
        f"Negative weights: {', '.join(top_features[lbl]['negative'])}",
        styles["Normal"]
    ))
    story.append(Spacer(1,10))

pdf = SimpleDocTemplate(pdf_path, pagesize=A4)
pdf.build(story)

print("✓ FULL PDF REPORT SUCCESS!")
print("Saved to:", pdf_path)


✓ FULL PDF REPORT SUCCESS!
Saved to: ./sentiment_analysis_full_report.pdf
